## The following section is for Colab Users.
### Just run the following code cells

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 18.0 MB/s eta 0:00:00


In [2]:
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar -P /content/spark-3.0.2-bin-hadoop2.7/jars/
!cp /content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar /content/spark-3.0.2-bin-hadoop2.7/graphframes-0.8.2-spark3.0-s_2.12.zip

--2023-05-02 11:01:14--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 54.230.18.27, 54.230.18.120, 54.230.18.61, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|54.230.18.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247882 (242K) [binary/octet-stream]
Saving to: ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.03s   

2023-05-02 11:01:14 (7.60 MB/s) - ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’ saved [247882/247882]



In [3]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [4]:
import findspark
findspark.init()

In [5]:
!export PYSPARK_SUBMIT_ARGS="--master local[*] pyspark-shell"
!export PYSPARK_DRIVER_PYTHON=jupyter
!export PYSPARK_DRIVER_PYTHON_OPTS=notebook

In [6]:
from pyspark.sql import SparkSession
from graphframes import *

spark = SparkSession.builder.master("local[*]").appName("GraphFrames").getOrCreate()

In [7]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

**************************************************************************
**************************************************************************
**************************************************************************

In [8]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read departuredelays.csv in Edge DataFrame
### Read airport-codes-na.txt in Vertix DataFrame (the separator is Tab i.e sep = '\t' )

In [9]:
edge_df = spark.read.csv("/content/departuredelays.csv", header=True, inferSchema=True)

In [10]:
edge_df.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 5 rows



In [11]:
vertix_df = spark.read.csv("/content/airport-codes-na.txt", header=True, inferSchema=True,sep="\t")

In [12]:
vertix_df.show(5)

+----------+-----+-------+----+
|      City|State|Country|IATA|
+----------+-----+-------+----+
|Abbotsford|   BC| Canada| YXX|
|  Aberdeen|   SD|    USA| ABR|
|   Abilene|   TX|    USA| ABI|
|     Akron|   OH|    USA| CAK|
|   Alamosa|   CO|    USA| ALS|
+----------+-----+-------+----+
only showing top 5 rows



#### The US flight delays data set has five columns:
- The <b>date</b> column contains an integer like 02190925 . When converted, this maps to 02-19 09:25 am.
- The <b>delay</b> column gives the delay in minutes between the scheduled and actual departure times. Early departures show negative numbers.
- The <b>distance</b> column gives the distance in miles from the origin airport to the destination airport.
- The <b>origin</b> column contains the origin IATA airport code.
- The <b>destination</b> column contains the destination IATA airport code.

#### The airport-codes data set has four columns:
- The <b>IATA</b> column contains IATA airport code.
- The <b>City, State, and Country</b> columns contains information about the airport location. 

### In the vertix DataFrame, drop any duplicated rows with the same  IATA code.

In [13]:
vertix_df = vertix_df.dropDuplicates(subset=["IATA"])

In [14]:
vertix_df.show(5)

+-------------------+-----+-------+----+
|               City|State|Country|IATA|
+-------------------+-----+-------+----+
|         Binghamton|   NY|    USA| BGM|
|            Lebanon|   NH|    USA| LEB|
|           Montreal|   PQ| Canada| YUL|
|         Dillingham|   AK|    USA| DLG|
|International Falls|   MN|    USA| INL|
+-------------------+-----+-------+----+
only showing top 5 rows



### In the edges DataFrame:
- Rename the <b>date</b> columns to become <b>tripid</b>.
- Rename the <b>origin</b> columns to become <b>src</b>.
- Rename the <b>destination</b> columns to become <b>dst</b>.

In [15]:
edge_df = edge_df.withColumnRenamed("date", "tripid")
edge_df = edge_df.withColumnRenamed("origin", "src")
edge_df = edge_df.withColumnRenamed("destination", "dst")

In [16]:
edge_df.show(5)

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1011245|    6|     602|ABE|ATL|
|1020600|   -8|     369|ABE|DTW|
|1021245|   -2|     602|ABE|ATL|
|1020605|   -4|     602|ABE|ATL|
|1031245|   -4|     602|ABE|ATL|
+-------+-----+--------+---+---+
only showing top 5 rows



### In the Vertix DataFrame:
- Rename the <b>IATA</b> columns to become <b>id</b>.

In [17]:
vertix_df = vertix_df.withColumnRenamed("IATA", "id")

### Create GraphFrame from Vertix and Edges DataFrames

In [18]:
from graphframes import *
gf = GraphFrame(vertix_df,edge_df)

In [19]:
gf.vertices.show()

+-------------------+-----+-------+---+
|               City|State|Country| id|
+-------------------+-----+-------+---+
|         Binghamton|   NY|    USA|BGM|
|            Lebanon|   NH|    USA|LEB|
|           Montreal|   PQ| Canada|YUL|
|         Dillingham|   AK|    USA|DLG|
|International Falls|   MN|    USA|INL|
|         Wolf Point|   MT|    USA|OLF|
|        New Orleans|   LA|    USA|MSY|
|            Toronto|   ON| Canada|YTO|
|            Spokane|   WA|    USA|GEG|
|              Havre|   MT|    USA|HVR|
|            Burbank|   CA|    USA|BUR|
|      Orange County|   CA|    USA|SNA|
|             Dryden|   ON| Canada|YHD|
|         Fort Dodge|   IA|    USA|FOD|
|          Green Bay|   WI|    USA|GRB|
|        Great Falls|   MT|    USA|GTF|
|              Homer|   AK|    USA|HOM|
|        Idaho Falls|   ID|    USA|IDA|
|      Sioux Lookout|   ON| Canada|YXL|
|       Grand Rapids|   MI|    USA|GRR|
+-------------------+-----+-------+---+
only showing top 20 rows



In [20]:
gf.edges.show()

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1011245|    6|     602|ABE|ATL|
|1020600|   -8|     369|ABE|DTW|
|1021245|   -2|     602|ABE|ATL|
|1020605|   -4|     602|ABE|ATL|
|1031245|   -4|     602|ABE|ATL|
|1030605|    0|     602|ABE|ATL|
|1041243|   10|     602|ABE|ATL|
|1040605|   28|     602|ABE|ATL|
|1051245|   88|     602|ABE|ATL|
|1050605|    9|     602|ABE|ATL|
|1061215|   -6|     602|ABE|ATL|
|1061725|   69|     602|ABE|ATL|
|1061230|    0|     369|ABE|DTW|
|1060625|   -3|     602|ABE|ATL|
|1070600|    0|     369|ABE|DTW|
|1071725|    0|     602|ABE|ATL|
|1071230|    0|     369|ABE|DTW|
|1070625|    0|     602|ABE|ATL|
|1071219|    0|     569|ABE|ORD|
|1080600|    0|     369|ABE|DTW|
+-------+-----+--------+---+---+
only showing top 20 rows



### Determine the number of airports

In [21]:
gf.vertices.count()

524

### Determine the number of trips 

In [22]:
gf.edges.count()

436996

### What is the longest delay?

In [23]:
from pyspark.sql.functions import max
gf.edges.select(max('delay')).show()

+----------+
|max(delay)|
+----------+
|      1560|
+----------+



### Find out the number of delayed flights vs. early flights (flights that departed before actual time)

In [24]:
from pyspark.sql.functions import col , count
gf.edges.select('*')\
.where(col('delay') > 0)\
.agg(count(col('delay')).alias("Delayed Flights"))\
.show()

+---------------+
|Delayed Flights|
+---------------+
|         195955|
+---------------+



In [25]:
from pyspark.sql.functions import col , count
gf.edges.select('*')\
.where(col('delay') < 0)\
.agg(count(col('delay')).alias("Early Flights"))\
.show()

+-------------+
|Early Flights|
+-------------+
|       191724|
+-------------+



### What flight destinations departing SFO are most likely to have significant delays? Select the top 10
#### Hint: you should get the average delay for each destination for trips that depart from SFO only

In [26]:
from pyspark.sql.functions import col , avg ,desc

gf.edges.select('*')\
.where(col('src') == 'SFO')\
.groupby('dst')\
.agg(avg(col('delay')).alias("Average Delay"))\
.orderBy(desc('Average Delay'))\
.limit(10)\
.show()

+---+------------------+
|dst|     Average Delay|
+---+------------------+
|KOA|24.903225806451612|
|SUN|24.451612903225808|
|MDW|              24.1|
|COS| 24.06451612903226|
|HNL|23.746575342465754|
|MCI| 22.37313432835821|
|MSY|21.074074074074073|
|OGG|19.557142857142857|
|ANC|              17.0|
|ASE|16.976744186046513|
+---+------------------+



### Find the Incoming connections to the airport sorted in Desc. order.

In [27]:
gf.edges.select('*')\
.where(col('src') == 'SFO')\
.orderBy(desc('tripid'))\
.show()

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1312356|   -3|    1859|SFO|ATL|
|1312352|   20|    1604|SFO|ORD|
|1312345|   28|    1859|SFO|ATL|
|1312337|   -3|    2229|SFO|EWR|
|1312330|   -4|    2247|SFO|JFK|
|1312330|   -9|    1604|SFO|ORD|
|1312328|    0|     167|SFO|RNO|
|1312320|   -5|    2247|SFO|JFK|
|1312310|    0|    2245|SFO|FLL|
|1312302|   -5|     316|SFO|ONT|
|1312300|   -8|      74|SFO|SMF|
|1312255|   16|    1604|SFO|ORD|
|1312250|   -1|    2350|SFO|BOS|
|1312245|   31|    2102|SFO|IAD|
|1312245|   -2|    2245|SFO|FLL|
|1312245|    1|     388|SFO|SAN|
|1312243|   11|     478|SFO|PDX|
|1312242|    6|     286|SFO|MFR|
|1312240|   -2|    1806|SFO|DTW|
|1312235|    5|    2229|SFO|EWR|
+-------+-----+--------+---+---+
only showing top 20 rows



### Find the Outgoing connections from the airport sorted in Desc. order.

In [28]:
gf.edges.select('*')\
.where(col('dst') == 'SFO')\
.orderBy(desc('tripid'))\
.show()

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1312225|    3|     293|LAX|SFO|
|1312220|   -3|    2032|OGG|SFO|
|1312214|   -1|    2102|IAD|SFO|
|1312200|    7|    2057|KOA|SFO|
|1312155|    1|    2084|HNL|SFO|
|1312152|    0|     840|DEN|SFO|
|1312125|   12|     590|SEA|SFO|
|1312120|   52|    1273|DFW|SFO|
|1312115|    4|     566|PHX|SFO|
|1312110|   25|     293|LAX|SFO|
|1312105|    1|    1421|IAH|SFO|
|1312105|   -6|     228|SBA|SFO|
|1312105|   71|     840|DEN|SFO|
|1312055|   71|     388|SAN|SFO|
|1312045|   14|     360|LAS|SFO|
|1312045|   -4|     360|LAS|SFO|
|1312041|    5|    1859|ATL|SFO|
|1312040|   -4|     293|LAX|SFO|
|1312035|   -6|    2246|MIA|SFO|
|1312035|   -8|    2191|PHL|SFO|
+-------+-----+--------+---+---+
only showing top 20 rows



### Use motif finding to answer this question: which delays could we blame on SFO?
#### Hint: this practically means that SFO is a transit station

In [29]:
gf_sub_1 = GraphFrame(vertix_df,edge_df.limit(10000)).dropIsolatedVertices()

In [ ]:
transit = gf_sub_1.find("(v1)-[]->(v2); (v2)-[]->(v3); (v3)-[]->(v1)").filter("v1.id = 'SFO'").select("v1", "v3")
transit.show()

### Determine Airport Ranking in Desc. order using PageRank algorithm

In [ ]:
results = gf_sub_1.pageRank(maxIter=10)

pagerank_df = results.vertices.sort("pagerank", ascending=False)

pagerank_df.select("id", "pagerank").show(10)

KeyboardInterrupt: ignored

## Determine the most popular flights (single city hops)

In [ ]:
gf_sub_2 = GraphFrame(vertix_df,edge_df.limit(50000)).dropIsolatedVertices()

In [ ]:
from graphframes import *
from pyspark.sql.functions import col


single_city_hops = gf_sub_2.find("(src)-[flight]->(dst)")\
                    .filter("src.City == dst.City")

popular_flights = single_city_hops.groupBy("flight.src", "flight.dst")\
                    .count()\
                    .withColumnRenamed("count", "Occurrences")\
                    .orderBy(col("Occurrences").desc())

popular_flights.show(10)

KeyboardInterrupt: ignored

### Find and Save a Subragph that obtained from the following pattern:
#### The flight starts from an airport and return back to the same airport through 2 other airports.

In [ ]:
subgraph = gf_sub_2.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)")
subgraph.vertices.write.parquet("vertices",mode='overwrite')
subgraph.edges.write.parquet("edges",mode='overwrite')